In [56]:
from scrapy import signals
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
import scrapy
import multiprocessing
import urllib

In [57]:
class CrawlerWorker(multiprocessing.Process):

    def __init__(self, spider, result_queue):
        multiprocessing.Process.__init__(self)
        self.result_queue = result_queue

        self.crawler = CrawlerProcess(get_project_settings())
        #if not hasattr(project, 'crawler'):
        #    self.crawler.install()
        #self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)
        

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.result_queue.put(self.items)

In [58]:
class FreeozLinkSpider(scrapy.Spider):
    name = "FreeozLinkSpider"
    allowed_domains = ["www.freeoz.org"]
    start_urls = ['http://www.freeoz.org/ibbs/forum-7068-3.html']
    
    def __init__(self, *a, **kw):
        super(FreeozLinkSpider, self).__init__(*a, **kw)
        dispatcher.connect(self.spider_closed, signals.spider_closed)
        
        self.syd_file=open('sydney.link','w')
        self.mel_file=open('melbourne.link','w')
        self.act_file=open('canberra.link','w')
        self.per_file=open('perth.link','w')
        self.ade_file=open('adelaide.link','w')
        self.bne_file=open('brisbane.link','w')
        
    def spider_closed(self):
        self.syd_file.close()
        self.mel_file.close()
        self.bne_file.close()
        self.ade_file.close()
        self.per_file.close()
        self.act_file.close()

    def write_link_to_files(self,location,full_link):
        if location.encode('utf-8') == '悉尼':
                self.syd_file.write(full_link+'\n')
                return
        if location.encode('utf-8') == '墨尔本':
                self.mel_file.write(full_link+'\n')
                return
        if location.encode('utf-8') == '布里斯班':
                self.bne_file.write(full_link+'\n')
                return
        if location.encode('utf-8') == '阿德莱德':
                self.ade_file.write(full_link+'\n')
                return
        if location.encode('utf-8') == '佩斯':
                self.per_file.write(full_link+'\n')
                return
        if location.encode('utf-8') == '堪培拉':
                self.act_file.write(full_link+'\n')
                return
        print("get nothing.nothing match!!!!!!")

    def parse(self, response):
        th_tags=response.xpath('//th[@class="new"]')
        for th_tag in th_tags:
            location=th_tag.xpath('em/a/text()').extract()[0]
            link=th_tag.xpath('a[@onclick="atarget(this)"]/@href').extract()[0]
            full_link=urllib.basejoin(response.url,link)
            self.write_link_to_files(location,full_link)

In [59]:
def main():
    result_queue = Queue()
    crawler = CrawlerWorker(FreeozLinkSpider(), result_queue)
    crawler.start()
    for item in result_queue.get():
        #print datetime.datetime.now(),item
        print item

In [60]:
if __name__ == '__main__':
    main()

2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 16:11:07 [scrapy] INFO: Overridden settings: {}
2017-01-23 16:11:07 [scrapy] INFO: Overridden settings: {}
2017-01-23 16:11:07 [scrapy] INFO: Overridden settings: {}
2017-01-23 16:11:07 

get nothing.nothing match!!!!!!


2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Closing spider (finished)
2017-01-23 16:11:16 [scrapy] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 234,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 19339,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'finish_reason': 'finished',
 'finish_time': dat